Case Studies

Project: 1

Group: 3

Group Members:
 - Muhammad Raafey Tariq (231806)
 - Farrukh Ahmed (230614)
 - Amirreza Khamehchin Khiabani (230891)
 - Aymane Hachcham (236392)


Requirements:
 - numpy==1.24.2
 - matplotlib==3.7.1
 - seaborn==0.12.2
 - pandas==2.0.0
 - openpyxl==3.1.2

Installation Commands (One-time only)
 - pip install pandas==2.0.0
 - pip install numpy==1.24.2
 - pip install seaborn==0.12.2
 - pip install matplotlib==3.7.1
 - pip install openpyxl==3.1.2

Imports and Libraries

In [135]:
import pandas as pd
import numpy as np
import itertools
import pprint
import random

# used for the graphs
import seaborn as sns

import os
sns.set(font_scale = 1.2)

# used for plotting
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


import matplotlib

# setting font to 'Times New Roman'
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams.update({'font.size': 16})
%matplotlib inline

Global Variables and Constants

Importing Data

In [136]:
file_path = "../styrian_health_data.xlsx"
sheet_name = "Sheet 1"
data_df = pd.read_excel(file_path, sheet_name=sheet_name)

c:\Users\Raafe\AppData\Local\Programs\Python\Python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Reformating Columns to Correct Data Types and dropping nans

In [137]:
# function to format variable types, remove nans, shuffle data
def format_variables(data, to_filter=[]):
    data_df = data.copy()
    data_df.postleitzahl = data_df.postleitzahl.astype('str')
    data_df.geburtsjahr = data_df.geburtsjahr.astype('Int64')
    # data_df.befinden = data_df.befinden.astype('Int64')
    data_df.messwert_bp_sys = pd.to_numeric(data_df.messwert_bp_sys)
    data_df.messwert_bp_dia = pd.to_numeric(data_df.messwert_bp_dia)
    data_df.schaetzwert_bp_sys = pd.to_numeric(data_df.schaetzwert_bp_sys)
    data_df.schaetzwert_by_dia = pd.to_numeric(data_df.schaetzwert_by_dia)

    # adding variable for is_local
    mask = data_df.gemeinde.isna() & data_df.bezirk.isna() & data_df.bundesland.isna()
    data_df["is_local_resident"] = True
    data_df.loc[mask, "is_local_resident"] = False

    # adding variable for age
    age =  data_df["zeit"].dt.year - pd.to_datetime(data_df['geburtsjahr'], format='%Y').dt.year
    data_df["age"] = age.astype("Int64")

    # adding variable for age group
    data_df["age_group"] = pd.cut(data_df.age, bins=[0,12,19,65,130],labels=['children', 'teenager', 'adult','65 over'])
    data_df["age_group"] = data_df.age_group.astype(str)

    #replacing nans for variables

    data_df.loc[data_df.geschlecht.isna() == True, 'raucher'] = "unknown"
    data_df.loc[data_df.geschlecht.isna() == True, 'blutzucker_bekannt'] = "unknown"
    data_df.loc[data_df.geschlecht.isna() == True, 'cholesterin_bekannt'] = "unknown"
    data_df.loc[data_df.geschlecht.isna() == True, 'in_behandlung'] = "unknown"
    data_df.loc[data_df.geschlecht.isna() == True, 'befinden'] = "unknown"
    data_df.loc[data_df.age_group == "nan", 'age_group'] = "unknown"

    data_df.loc[mask, 'gemeinde'] = "not applicable"
    data_df.loc[mask, 'bezirk'] = "not applicable"
    data_df.loc[mask, 'bundesland'] = "not applicable"
    data_df.loc[mask, 'postleitzahl'] = "not applicable"
    data_df.loc[data_df.postleitzahl == "nan", 'postleitzahl'] = "unknown"

    # creating variables for missing values in bp

    data_df["is_missing_schaetzwert_bp_sys"] = False
    data_df.loc[data_df.schaetzwert_bp_sys.isna() == True, "is_missing_schaetzwert_bp_sys"] = True
    data_df["is_missing_schaetzwert_by_dia"] = False
    data_df.loc[data_df.schaetzwert_by_dia.isna() == True, "is_missing_schaetzwert_by_dia"] = True

    # removing useless variables
    data_df.drop(data_df[data_df.age > 100].index, inplace=True)
    data_df.drop(data_df[data_df.age < 15].index, inplace=True)

    data_df.loc[data_df.geschlecht.isna() == True, 'geschlecht'] = "unknown"

    data_df = data_df.dropna()

    if len(to_filter) > 0:
        data_df = data_df.drop(to_filter, axis=1)

    data_df.befinden = data_df.befinden.astype('Int64')

    data_df['befinden'] = data_df['befinden'].astype(object)
    data_df['messwert_bp_sys'] = data_df['messwert_bp_sys'].astype(float)
    data_df['messwert_bp_dia'] = data_df['messwert_bp_dia'].astype(float)
    data_df['geschlecht'] = data_df['geschlecht'].astype(object)
    data_df['is_local_resident'] = data_df['is_local_resident'].astype(object)

    # shuffling data with fixed seed
    data_df = data_df.sample(frac=1, random_state=1).reset_index(drop=True)

    # separating var types
    cat_feat_list = []
    num_feat_list = []

    for var in data_df.columns:
        if data_df[var].dtype == object:
            cat_feat_list.append(var)
        else:
            num_feat_list.append(var)

    return data_df, cat_feat_list, num_feat_list


# function that converts cat columns in df to one-hot encoding
def encode_data(df, cat_feat_list, num_feat_list):
    one_hot_data = pd.get_dummies(df[cat_feat_list], drop_first=True, dtype=int)

    for var in num_feat_list:
        one_hot_data[var] = df[var] 
    
    return one_hot_data

# function to separate target from dataframe
def separate_target(data, target):
    df = data.copy()
    Y = df[target]
    del df[target]
    X = df

    return X, Y

# computes adjusted R2
def adjusted_r2(r_2, n, k):
    return 1 - (1-r_2)*(n-1)/(n-k-1)


# function to compute metrics given target and predictions
def compute_metrics(pred, target, num_feats):
    r_2 = r2_score(pred, target)
    mse = mean_squared_error(pred, target)
    adj_r2 = adjusted_r2(r_2, len(pred), num_feats)
    return {
        "r_2": r_2,
        "adjusted_r_2": adj_r2,
        "mse": mse
    }

# method that fits and predicts regression tree based on model type
def fit_and_eval_regression_tree(X_train, Y_train, X_test, params, model_type):
    model = None
    if model_type == "DecisionTreeRegressor":
        model = DecisionTreeRegressor(**params)
    elif model_type == "DecisionTreeRegressorRandomForest":
        model = RandomForestRegressor(**params)

    model.fit(X_train, Y_train)
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    return train_predictions, test_predictions, model

# method that fits, predicts, generates eval metrics for regression tree based on model type
def fit_model(X_train, Y_train, X_test, Y_test, model_type, params):
    num_feats = len(X_train.columns)
    train_predictions = None
    train_predictions = None
    train_results = None
    test_results = None
    model = None

    if model_type in ["DecisionTreeRegressor", "DecisionTreeRegressorRandomForest"]:
        train_predictions, test_predictions, model = fit_and_eval_regression_tree(X_train, Y_train, X_test, params, model_type)

    if train_predictions is not None and test_predictions is not None:
        train_results = compute_metrics(train_predictions, Y_train, num_feats)
        test_results = compute_metrics(test_predictions, Y_test, num_feats)


    return train_results, test_results, model

# method that performs best subset feat selection based on some creiterion like mse, adjusted_r_2 or r_2
def best_subset_selection(features, criterion, X_train, Y_train, X_test, Y_test, model_type, params, verbose):
    if criterion == "mse":
        best_val = np.inf
    elif criterion in ["adjusted_r_2", "r_2"]:
        best_val = -np.inf

    best_train_results = None
    best_model = None
    best_test_results = None
    best_features = None
    n_features = len(features)


    for i in range(1, n_features):
        if verbose > 1:
            print("\nNum features: ", i, "=======================================================")

        for j in range(n_features):
            current_features = features[j:j+i]
            if len(current_features) < i:
                break

            X_train_curr = X_train[current_features]
            X_test_curr = X_test[current_features]
            
            train_results, test_results, model = fit_model(X_train_curr, Y_train, X_test_curr, Y_test, model_type, params)

            if verbose > 1:
                print("\nFeatures: ", current_features)
                print("Train Results: ", train_results)
                print("Test Results: ", test_results)

            condition = False
            if criterion == "mse":
                condition = test_results[criterion] < best_val
            elif criterion in ["adjusted_r_2", "r_2"]:
                condition = test_results[criterion] > best_val   

            if condition:
                best_val = test_results[criterion]
                best_model = model
                best_features = current_features
                best_train_results = train_results
                best_test_results = test_results
    
    if verbose > 0:
        print("\nBest Model: ")
        print("Features: ", best_features)
        print("Train Results: ", best_train_results)
        print("Test Results: ", best_test_results)
    
    return best_model, best_train_results, best_test_results

# method that formats results of different models in a dataframe
def tabularize_model_metrics(train_result_list, test_result_list, model_names):
    train_df = pd.DataFrame(train_result_list)
    test_df = pd.DataFrame(test_result_list)
    train_df = train_df.rename(columns={"adjusted_r_2": "Train Adjusted R2", "r_2": "Train R2", "mse": "Train Mean Sq Error"})
    test_df = test_df.rename(columns={"adjusted_r_2": "Test Adjusted R2", "r_2": "Test R2", "mse": "Test Mean Sq Error"})
    df = pd.concat([train_df, test_df], axis=1)
    df["Model"] = model_names
    return df[["Model", "Train Mean Sq Error", "Test Mean Sq Error", "Train R2", "Test R2", "Train Adjusted R2", "Test Adjusted R2"]]


In [138]:
# variables that are dropped 
to_filter = ["id", "zeit", "postleitzahl", "gemeinde", "bezirk", "geburtsjahr",
              "is_missing_schaetzwert_bp_sys", "is_missing_schaetzwert_by_dia", "terminal"]
data_df, cat_feat_list, num_feat_list = format_variables(data_df, to_filter=to_filter)

# one hot encoding cat variables to prep data for Decision Tree
# ordinal variables and nominal are treated the same in trees, but need to be careful in Lin models

encoded_data_df = encode_data(data_df, cat_feat_list, num_feat_list)
encoded_train_set, encoded_test_set = train_test_split(encoded_data_df, test_size=0.3)

In [139]:
print("Size of training data: ", len(encoded_train_set))
print("Size of testing data: ", len(encoded_test_set))
print("Features used: ", data_df.columns)

Size of training data:  10381
Size of testing data:  4450
Features used:  Index(['bundesland', 'befinden', 'geschlecht', 'raucher', 'blutzucker_bekannt',
       'cholesterin_bekannt', 'in_behandlung', 'schaetzwert_bp_sys',
       'schaetzwert_by_dia', 'messwert_bp_sys', 'messwert_bp_dia',
       'is_local_resident', 'age', 'age_group'],
      dtype='object')


https://scikit-learn.org/stable/modules/tree.html#tree

 - scikit-learn uses an optimized version of the CART algorithm, does not support categorical variables
 - BIC cannot be computed as it depends on likelihood, cannot compute that for RegressionTree as it does not assume a conditional dist of data

In [140]:
# using self evaluated sys bp for analysis
target = "messwert_bp_sys"

# splitting targets from predictors
X_train, Y_train = separate_target(encoded_train_set, target)
X_test, Y_test = separate_target(encoded_test_set, target)

In [141]:
# fitting base model for DecisonTreeRegressor using all available features and default parameters

train_results_tree_base, test_results_tree_base, model_tree_base = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressor", {"criterion" : "squared_error"})

In [142]:
# fitting base model for RandomForestRegressor using all available features and default parameters

train_results_rf_base, test_results_rf_base, model_rf_base = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressorRandomForest", {"criterion" : "squared_error",
                                                                                                                    "n_estimators": 100})

In [143]:
# finding the best set of parameters to use by finetuning RegTree using CV, fine-tuning is done on whole dataset

parameters= {"splitter":["best","random"],
            "max_depth" : list(np.arange(1, 25, 5, dtype=int)),
           "min_samples_leaf":list(np.arange(1, 100, 5, dtype=int)),
           "min_weight_fraction_leaf":list(np.arange(0, 1, 1.0, dtype=float)),
           "max_features":["auto","log2","sqrt", None]
           }

model = DecisionTreeRegressor()
X_train_full, Y_train_full = separate_target(encoded_data_df, target)
tuning_model = GridSearchCV(model, param_grid=parameters, scoring='neg_mean_squared_error',cv=10,verbose=0)
tuning_model.fit(X_train_full, Y_train_full)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [1, 6, 11, 16, 21],
                         'max_features': ['auto', 'log2', 'sqrt', None],
                         'min_samples_leaf': [1, 6, 11, 16, 21, 26, 31, 36, 41,
                                              46, 51, 56, 61, 66, 71, 76, 81,
                                              86, 91, 96],
                         'min_weight_fraction_leaf': [0.0],
                         'splitter': ['best', 'random']},
             scoring='neg_mean_squared_error')

In [144]:
tuning_model.best_params_

{'max_depth': 11,
 'max_features': 'auto',
 'min_samples_leaf': 66,
 'min_weight_fraction_leaf': 0.0,
 'splitter': 'best'}

In [145]:
# fitting fine_tuned model for DecisonTreeRegressor using all available features and fine_tuned parameters
model_params = tuning_model.best_params_.copy()
model_params["criterion"] = "squared_error"

train_results_tree_fine, test_results_tree_fine, model_tree_fine = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressor",
                                                                              model_params)

In [146]:
# fitting fine_tuned model for RandomForestRegressor using all available features and fine_tuned parameters
model_params = tuning_model.best_params_.copy()
model_params["criterion"] = "squared_error"
del model_params["splitter"]
model_params["n_estimators"] = 100

train_results_rf_fine, test_results_rf_fine, model_rf_fine = fit_model(X_train, Y_train, X_test, Y_test,
                                                                              "DecisionTreeRegressorRandomForest",
                                                                              model_params)

In [147]:
# setting criterion for best subset selection
BEST_SUBSET_CRITERION = "mse"

In [148]:
# using best subset selection with default parameters for DecisionTreeRegressor

model_type = "DecisionTreeRegressor"
model_params = {}
model_params["criterion"] = "squared_error"
criterion = BEST_SUBSET_CRITERION
features = list(X_train.columns)

model_tree_base_best, train_results_tree_base_best, test_results_tree_base_best = best_subset_selection(features, criterion, X_train, Y_train, X_test, Y_test,
                                                     model_type, model_params, 1)


Best Model: 
Features:  ['messwert_bp_dia']
Train Results:  {'r_2': -0.19055570853385229, 'adjusted_r_2': -0.19067041666647921, 'mse': 202.3280427992853}
Test Results:  {'r_2': -0.2710051773240114, 'adjusted_r_2': -0.2712909248908557, 'mse': 208.34464192287473}


In [149]:
# using best subset selection with default parameters for DecisionTreeRegressorRandomForest
model_params = {}
model_params["criterion"] = "squared_error"
model_params["n_estimators"] = 100
model_type = "DecisionTreeRegressorRandomForest"
criterion = BEST_SUBSET_CRITERION
features = list(X_train.columns)
model_rf_base_best, train_results_rf_base_best, test_results_rf_base_best = best_subset_selection(features, criterion, X_train, Y_train, X_test, Y_test,
                                                     model_type, model_params, 1)


Best Model: 
Features:  ['bundesland_Vorarlberg', 'bundesland_Wien', 'bundesland_not applicable', 'befinden_2', 'befinden_3', 'befinden_4', 'befinden_5', 'geschlecht_m', 'raucher_True', 'blutzucker_bekannt_True', 'cholesterin_bekannt_True', 'in_behandlung_True', 'is_local_resident_True', 'age_group_adult', 'age_group_teenager', 'schaetzwert_bp_sys', 'schaetzwert_by_dia', 'messwert_bp_dia', 'age']
Train Results:  {'r_2': 0.9036617935659197, 'adjusted_r_2': 0.9034851285796975, 'mse': 26.60463208309184}
Test Results:  {'r_2': 0.18024607732305697, 'adjusted_r_2': 0.17673020271112416, 'mse': 183.376144932512}


In [150]:
# using best subset selection with finetuned parameters for DecisionTreeRegressor

model_type = "DecisionTreeRegressor"
model_params = tuning_model.best_params_.copy()
model_params["criterion"] = "squared_error"
criterion = BEST_SUBSET_CRITERION
features = list(X_train.columns)

model_tree_fine_best, train_results_tree_fine_best, test_results_tree_fine_best = best_subset_selection(features, criterion, X_train, Y_train, X_test, Y_test,
                                                     model_type, model_params, 1)


Best Model: 
Features:  ['schaetzwert_bp_sys', 'schaetzwert_by_dia', 'messwert_bp_dia', 'age']
Train Results:  {'r_2': 0.1913485408422585, 'adjusted_r_2': 0.19103680165214376, 'mse': 166.44361989036423}
Test Results:  {'r_2': 0.12538164915665018, 'adjusted_r_2': 0.12459459102315784, 'mse': 178.33355743654016}


In [151]:
# using best subset selection with finetuned parameters for DecisionTreeRegressorRF

model_type = "DecisionTreeRegressorRandomForest"
model_params = tuning_model.best_params_.copy()
del model_params["splitter"]
model_params["n_estimators"] = 100
criterion = BEST_SUBSET_CRITERION
features = list(X_train.columns)
model_rf_fine_best, train_results_rf_fine_best, test_results_rf_fine_best = best_subset_selection(features, criterion, X_train, Y_train, X_test, Y_test,
                                                     model_type, model_params, 1)


Best Model: 
Features:  ['befinden_3', 'befinden_4', 'befinden_5', 'geschlecht_m', 'raucher_True', 'blutzucker_bekannt_True', 'cholesterin_bekannt_True', 'in_behandlung_True', 'is_local_resident_True', 'age_group_adult', 'age_group_teenager', 'schaetzwert_bp_sys', 'schaetzwert_by_dia', 'messwert_bp_dia', 'age']
Train Results:  {'r_2': 0.16916897471651082, 'adjusted_r_2': 0.1679666143325984, 'mse': 163.75304239926163}
Test Results:  {'r_2': 0.11222288545170755, 'adjusted_r_2': 0.10921957992211251, 'mse': 173.14674199095273}


In [154]:
train_result_list = [train_results_tree_base, train_results_tree_fine, train_results_tree_base_best, train_results_tree_fine_best,
                     train_results_rf_base, train_results_rf_fine, train_results_rf_base_best, train_results_rf_fine_best]

test_result_list = [test_results_tree_base, test_results_tree_fine, test_results_tree_base_best, test_results_tree_fine_best,
                     test_results_rf_base, test_results_rf_fine, test_results_rf_base_best, test_results_rf_fine_best]

model_names = ["Tree (Base)", "Tree (Fine-tuned)", "Tree (Best Subset + Base)", "Tree (Best Subset + Fine-tuned)",
               "RF (Base)", "RF (Fine-tuned)", "RF (Best Subset + Base)", "RF (Best Subset + Fine-tuned)"]

tab = tabularize_model_metrics(train_result_list, test_result_list, model_names)
round(tab, 3)

,Model,Train Mean Sq Error,Test Mean Sq Error,Train R2,Test R2,Train Adjusted R2,Test Adjusted R2
0,Tree (Base),1.476,361.268,0.996,0.107,0.996,0.102
1,Tree (Fine-tuned),164.102,178.981,0.212,0.133,0.210,0.128
2,Tree (Best Subset + Base),202.328,208.345,-0.191,-0.271,-0.191,-0.271
3,Tree (Best Subset + Fine-tuned),166.444,178.334,0.191,0.125,0.191,0.125
4,RF (Base),26.434,184.895,0.905,0.179,0.904,0.175
5,RF (Fine-tuned),163.389,173.029,0.168,0.110,0.166,0.105
6,RF (Best Subset + Base),26.605,183.376,0.904,0.180,0.903,0.177
7,RF (Best Subset + Fine-tuned),163.753,173.147,0.169,0.112,0.168,0.109
